In [ ]:
#default_exp data

In [ ]:
#export
import torch
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
import mat73
from fastcore.all import *
from fastdownload import FastDownload

## Dataset

In [ ]:
#export

# This comes from fastai.torch_core

def _fig_bounds(x):
    r = x//32
    return min(5, max(1,r))

@delegates(plt.Axes.imshow, keep=True, but=['shape', 'imlim'])
def show_image(im, ax=None, figsize=None, title=None, ctx=None, **kwargs):
    "Show a PIL or PyTorch image on `ax`."
    # Handle pytorch axis order
    if hasattrs(im, ('data','cpu','permute')):
        im = im.data.cpu()
        if im.shape[0]<5: im=im.permute(1,2,0)
    elif not isinstance(im,np.ndarray): im=array(im)
    # Handle 1-channel images
    if im.shape[-1]==1: im=im[...,0]

    ax = ifnone(ax,ctx)
    if figsize is None: figsize = (_fig_bounds(im.shape[0]), _fig_bounds(im.shape[1]))
    if ax is None: _,ax = plt.subplots(figsize=figsize)
    ax.imshow(im, **kwargs)
    if title is not None: ax.set_title(title)
    ax.axis('off')
    return ax

@delegates(plt.subplots)
def show_images(ims, nrows=1, ncols=None, titles=None, **kwargs):
    "Show all images `ims` as subplots with `rows` using `titles`."
    if ncols is None: ncols = int(math.ceil(len(ims)/nrows))
    if titles is None: titles = [None]*len(ims)
    axs = plt.subplots(nrows, ncols, **kwargs)[1].flat
    for im,t,ax in zip(ims, titles, axs): show_image(im, ax=ax, title=t)

## Dataloader

In [ ]:
#export

class NYUDataset:
    
    _titles=["img", "depth", "label"]
    
    def __init__(self, dataset_path='data/nyu_depth_v2_labeled.mat', tfms=None):
        data = mat73.loadmat(dataset_path)
        self.images = data["images"]
        self.depths = data["depths"]
        self.labels = data["labels"]
        
    def __getitem__(self, idx):
        timg = T.ToTensor()(self.images[..., idx])
        tdepth = torch.as_tensor(self.depths[..., idx])
        tlabel = torch.as_tensor(self.labels[None, :, :, idx].astype(np.int64))
        return (timg, tdepth, tlabel)
        
    def show_one(self, idx=0):
        show_images(self.__getitem__(idx), titles=self._titles)
    
    def __len__(self): return self.images.shape[-1]